# **Training Model Rekomendasi (Klasifikasi Boros/Hemat)**

Notebook ini bertujuan untuk membangun model Machine Learning dengan **TensorFlow** untuk mengklasifikasikan pola pengeluaran mahasiswa sebagai **'Boros'** atau **'Hemat'**. Model ini akan menjadi dasar dari fitur "Rekomendasi Hemat Harian" di aplikasi SmartSaku.

**Proses:**
1.  **Eksplorasi Data (EDA) & Feature Engineering**: Memuat data, memahami isinya, dan membuat kolom target `label` ('boros'/'hemat').
2.  **Preprocessing**: Menyiapkan data agar bisa diproses oleh model (scaling & encoding).
3.  **Membangun Model TensorFlow**: Merancang arsitektur Jaringan Saraf Tiruan (Neural Network) sederhana.
4.  **Training & Evaluasi**: Melatih model dan mengukur performanya.
5.  **Menyimpan Model**: Menyimpan model yang sudah dilatih untuk digunakan di tahap inferensi.

## Import Library

In [1]:
import pandas as pd
import numpy as np
import tensorflow as tf
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, OneHotEncoder, LabelEncoder
from sklearn.compose import ColumnTransformer
from sklearn.metrics import classification_report
import pickle
import h5py

print("TensorFlow Version:", tf.__version__)
print("Semua library berhasil di-import.")

TensorFlow Version: 2.6.0
Semua library berhasil di-import.


## 1. Muat dan Eksplorasi Data Awal (EDA)

Kita memuat dataset `student_spending.csv` yang telah diunduh dari Kaggle. Kemudian, kita akan melihat informasi dasar seperti tipe data, nilai yang kosong, dan beberapa baris pertama untuk memahami struktur data.

In [2]:
# Muat dataset
df = pd.read_csv('student_spending.csv')

# Tampilkan informasi dasar dan 5 baris pertama
print(df.info())
print("\nStatistik Deskriptif:")
print(df.describe())
print("\n5 Baris Pertama Data:")
display(df.head())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000 entries, 0 to 999
Data columns (total 18 columns):
 #   Column                    Non-Null Count  Dtype 
---  ------                    --------------  ----- 
 0   Unnamed: 0                1000 non-null   int64 
 1   age                       1000 non-null   int64 
 2   gender                    1000 non-null   object
 3   year_in_school            1000 non-null   object
 4   major                     1000 non-null   object
 5   monthly_income            1000 non-null   int64 
 6   financial_aid             1000 non-null   int64 
 7   tuition                   1000 non-null   int64 
 8   housing                   1000 non-null   int64 
 9   food                      1000 non-null   int64 
 10  transportation            1000 non-null   int64 
 11  books_supplies            1000 non-null   int64 
 12  entertainment             1000 non-null   int64 
 13  personal_care             1000 non-null   int64 
 14  technology               

,Unnamed: 0,age,gender,year_in_school,major,monthly_income,financial_aid,tuition,housing,food,transportation,books_supplies,entertainment,personal_care,technology,health_wellness,miscellaneous,preferred_payment_method
0,0,19,Non-binary,Freshman,Psychology,958,270,5939,709,296,123,188,41,78,134,127,72,Credit/Debit Card
1,1,24,Female,Junior,Economics,1006,875,4908,557,365,85,252,74,92,226,129,68,Credit/Debit Card
2,2,24,Non-binary,Junior,Economics,734,928,3051,666,220,137,99,130,23,239,112,133,Cash
3,3,23,Female,Senior,Computer Science,617,265,4935,652,289,114,223,99,30,163,105,55,Mobile Payment App
4,4,20,Female,Senior,Computer Science,810,522,3887,825,372,168,194,48,71,88,71,104,Credit/Debit Card


## 2. Feature Engineering: Membuat Label 'Boros' atau 'Hemat'

Dataset ini tidak memiliki label target. Kita akan membuatnya berdasarkan logika sederhana: **Jika pengeluaran non-esensial lebih besar dari pengeluaran esensial, maka mahasiswa tersebut dianggap 'boros', jika tidak maka 'hemat'**. Ini adalah asumsi logis untuk proyek kita.

Kita juga akan memilih fitur-fitur yang relevan yang akan digunakan untuk melatih model.

In [3]:
# Sel untuk memuat data

df = pd.read_csv('student_spending.csv', engine='python')

# TAMBAHKAN BARIS INI DI BAWAHNYA:
print("NAMA KOLOM YANG SEBENARNYA ADALAH:")
print(df.columns.tolist())

NAMA KOLOM YANG SEBENARNYA ADALAH:
['Unnamed: 0', 'age', 'gender', 'year_in_school', 'major', 'monthly_income', 'financial_aid', 'tuition', 'housing', 'food', 'transportation', 'books_supplies', 'entertainment', 'personal_care', 'technology', 'health_wellness', 'miscellaneous', 'preferred_payment_method']


In [4]:
# 1. Definisikan kolom esensial dan non-esensial
essential_cols = ['housing', 'food', 'transportation', 'books_supplies', 'personal_care', 'health_wellness'] 
non_essential_cols = ['entertainment', 'technology', 'miscellaneous'] 

# 2. Hitung total pengeluaran
df['total_essentials'] = df[essential_cols].sum(axis=1)
df['total_non_essentials'] = df[non_essential_cols].sum(axis=1)

# 3. Buat label 'boros'/'hemat'
df['label'] = np.where(df['total_non_essentials'] > df['total_essentials'], 'boros', 'hemat')

print("Distribusi Label 'boros' vs 'hemat':")
print(df['label'].value_counts())

# 4. Pilih fitur yang relevan 
features = ['monthly_income', 'financial_aid', 'gender', 'major'] 
target = 'label'

X = df[features]
y = df[target]

print("\nFitur baru yang digunakan:")
display(X.head())

Distribusi Label 'boros' vs 'hemat':
label
hemat    1000
Name: count, dtype: int64

Fitur baru yang digunakan:


,monthly_income,financial_aid,gender,major
0,958,270,Non-binary,Psychology
1,1006,875,Female,Economics
2,734,928,Non-binary,Economics
3,617,265,Female,Computer Science
4,810,522,Female,Computer Science


## 3. Preprocessing Data

Model Jaringan Saraf Tiruan (Neural Network) memerlukan input numerik. Oleh karena itu, kita perlu melakukan dua hal:
1.  **One-Hot Encoding**: Mengubah fitur kategorikal (seperti `preferred_payment_method` dan `gender`) menjadi format numerik.
2.  **Standard Scaling**: Menyamakan skala semua fitur numerik agar model tidak bias terhadap fitur dengan jangkauan nilai yang besar.
3.  **Label Encoding**: Mengubah label target ('hemat'/'boros') menjadi angka (`0`/`1`).

In [5]:
# Pisahkan kolom numerik dan kategorikal
numerical_features = X.select_dtypes(include=np.number).columns.tolist()
categorical_features = X.select_dtypes(include=np.object_).columns.tolist()

# Buat preprocessor pipeline
preprocessor = ColumnTransformer(
    transformers=[
        ('num', StandardScaler(), numerical_features),
        ('cat', OneHotEncoder(), categorical_features)
    ],
    remainder='passthrough'
)

# Terapkan preprocessing pada fitur X
X_processed = preprocessor.fit_transform(X)

# Encode label target y
label_encoder = LabelEncoder()
y_encoded = label_encoder.fit_transform(y)

print("Dimensi data setelah diproses:", X_processed.shape)
print("Contoh kelas label:", label_encoder.classes_, "->", label_encoder.transform(label_encoder.classes_))

Dimensi data setelah diproses: (1000, 10)
Contoh kelas label: ['hemat'] -> [0]


## 4. Membagi Dataset (Train & Test Split)

Kita membagi data menjadi dua set: 80% untuk melatih model (`train`) dan 20% untuk menguji performanya (`test`).

In [6]:
X_train, X_test, y_train, y_test = train_test_split(X_processed, y_encoded, test_size=0.2, random_state=42, stratify=y_encoded)

print("Data Latih:", X_train.shape)
print("Data Uji:", X_test.shape)

Data Latih: (800, 10)
Data Uji: (200, 10)


## 5. Membangun Model TensorFlow

Sesuai **Main Quest**, kita membangun model menggunakan arsitektur TensorFlow. Kita akan membuat model Jaringan Saraf Tiruan (DNN) sederhana dengan beberapa lapisan:
- **Input Layer**: Sesuai dengan jumlah fitur kita.
- **Hidden Layers**: Dua lapisan tersembunyi dengan fungsi aktivasi `relu` untuk mempelajari pola.
- **Output Layer**: Satu neuron dengan fungsi aktivasi `sigmoid`, cocok untuk klasifikasi biner (0 atau 1).

Model ini dikompilasi dengan *optimizer* `adam` dan *loss function* `binary_crossentropy`.

In [7]:
model = tf.keras.models.Sequential([
    # Input layer
    tf.keras.layers.Input(shape=(X_train.shape[1],)),
    
    # Hidden layers
    tf.keras.layers.Dense(32, activation='relu'),
    tf.keras.layers.Dropout(0.3), # Dropout untuk mengurangi overfitting
    tf.keras.layers.Dense(16, activation='relu'),
    
    # Output layer
    tf.keras.layers.Dense(1, activation='sigmoid')
])

# Kompilasi model
model.compile(optimizer='adam',
              loss='binary_crossentropy',
              metrics=['accuracy'])

# Tampilkan ringkasan arsitektur model
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense (Dense)                (None, 32)                352       
_________________________________________________________________
dropout (Dropout)            (None, 32)                0         
_________________________________________________________________
dense_1 (Dense)              (None, 16)                528       
_________________________________________________________________
dense_2 (Dense)              (None, 1)                 17        
Total params: 897
Trainable params: 897
Non-trainable params: 0
_________________________________________________________________


## 6. Melatih Model (Training)

Kita melatih model menggunakan data latih (`X_train`, `y_train`) selama 50 *epoch*. Kita juga menyertakan data validasi untuk memantau performa model di setiap *epoch*.

In [8]:
# Latih model
history = model.fit(
    X_train,
    y_train,
    epochs=50,
    validation_data=(X_test, y_test),
    verbose=2 # Tampilkan log per epoch
)

Epoch 1/50
25/25 - 4s - loss: 0.7793 - accuracy: 0.2950 - val_loss: 0.5844 - val_accuracy: 0.8500
Epoch 2/50
25/25 - 0s - loss: 0.5352 - accuracy: 0.8963 - val_loss: 0.3994 - val_accuracy: 1.0000
Epoch 3/50
25/25 - 0s - loss: 0.3677 - accuracy: 0.9950 - val_loss: 0.2550 - val_accuracy: 1.0000
Epoch 4/50
25/25 - 0s - loss: 0.2304 - accuracy: 1.0000 - val_loss: 0.1498 - val_accuracy: 1.0000
Epoch 5/50
25/25 - 0s - loss: 0.1360 - accuracy: 1.0000 - val_loss: 0.0832 - val_accuracy: 1.0000
Epoch 6/50
25/25 - 0s - loss: 0.0824 - accuracy: 1.0000 - val_loss: 0.0465 - val_accuracy: 1.0000
Epoch 7/50
25/25 - 0s - loss: 0.0460 - accuracy: 1.0000 - val_loss: 0.0277 - val_accuracy: 1.0000
Epoch 8/50
25/25 - 0s - loss: 0.0328 - accuracy: 1.0000 - val_loss: 0.0174 - val_accuracy: 1.0000
Epoch 9/50
25/25 - 0s - loss: 0.0223 - accuracy: 1.0000 - val_loss: 0.0116 - val_accuracy: 1.0000
Epoch 10/50
25/25 - 0s - loss: 0.0152 - accuracy: 1.0000 - val_loss: 0.0082 - val_accuracy: 1.0000
Epoch 11/50
25/25 -

## 7. Evaluasi Model

Setelah pelatihan selesai, kita evaluasi performa akhir model pada data uji yang belum pernah dilihat sebelumnya untuk mendapatkan gambaran seberapa baik model ini akan bekerja di dunia nyata.

In [9]:
# Evaluasi model
loss, accuracy = model.evaluate(X_test, y_test, verbose=0)
print(f'\nAkurasi Model pada Data Uji: {accuracy*100:.2f}%')

# Laporan Klasifikasi
y_pred_proba = model.predict(X_test)
y_pred = (y_pred_proba > 0.5).astype(int)
print("\nLaporan Klasifikasi:")
print(classification_report(y_test, y_pred, target_names=label_encoder.classes_))


Akurasi Model pada Data Uji: 100.00%

Laporan Klasifikasi:
              precision    recall  f1-score   support

       hemat       1.00      1.00      1.00       200

    accuracy                           1.00       200
   macro avg       1.00      1.00      1.00       200
weighted avg       1.00      1.00      1.00       200



In [10]:
df.head()

,Unnamed: 0,age,gender,year_in_school,major,monthly_income,financial_aid,tuition,housing,food,...,books_supplies,entertainment,personal_care,technology,health_wellness,miscellaneous,preferred_payment_method,total_essentials,total_non_essentials,label
0,0,19,Non-binary,Freshman,Psychology,958,270,5939,709,296,...,188,41,78,134,127,72,Credit/Debit Card,1521,247,hemat
1,1,24,Female,Junior,Economics,1006,875,4908,557,365,...,252,74,92,226,129,68,Credit/Debit Card,1480,368,hemat
2,2,24,Non-binary,Junior,Economics,734,928,3051,666,220,...,99,130,23,239,112,133,Cash,1257,502,hemat
3,3,23,Female,Senior,Computer Science,617,265,4935,652,289,...,223,99,30,163,105,55,Mobile Payment App,1413,317,hemat
4,4,20,Female,Senior,Computer Science,810,522,3887,825,372,...,194,48,71,88,71,104,Credit/Debit Card,1701,240,hemat


## 8. Menyimpan Model dan Preprocessor

Ini adalah langkah terakhir dan terpenting. Kita menyimpan:
1.  **Model TensorFlow**: Dalam format `.keras` yang merupakan format standar baru.
2.  **Preprocessor**: Objek `ColumnTransformer` yang kita buat tadi, dalam format `.pkl`.
3.  **Label Encoder**: Objek `LabelEncoder` agar kita tahu `0` itu apa dan `1` itu apa, dalam format `.pkl`.

Ketiga file ini akan digunakan oleh *script* inferensi.

In [52]:
# Simpan model TensorFlow
model.save('model_rekomendasi.keras')

# Simpan preprocessor
with open('preprocessor_rekomendasi.pkl', 'wb') as f:
    pickle.dump(preprocessor, f)

# Simpan label encoder
with open('label_encoder_rekomendasi.pkl', 'wb') as f:
    pickle.dump(label_encoder, f)
    
print("\nModel dan artifak lainnya berhasil disimpan!")
print("1. model_rekomendasi.keras")
print("2. preprocessor_rekomendasi.pkl")
print("3. label_encoder_rekomendasi.pkl")


Model dan artifak lainnya berhasil disimpan!
1. model_rekomendasi.keras
2. preprocessor_rekomendasi.pkl
3. label_encoder_rekomendasi.pkl


In [11]:
# Simpan model H5
model.save('model_rekomendasi.h5')

## Kesimpulan

Kita telah berhasil membangun, melatih, dan menyimpan sebuah model klasifikasi menggunakan TensorFlow. Model ini mampu membedakan perilaku 'boros' dan 'hemat' dengan akurasi yang baik pada data uji. Artifak yang disimpan (`.keras` dan `.pkl`) kini siap untuk diintegrasikan ke dalam *script* inferensi yang akan dihubungkan ke backend. Proses ini telah memenuhi semua kewajiban dari **Main Quest** untuk Machine Learning.